# 13. Volatility and IR Options

## Overview

* The most liquid nonlinear OCT rates products are SOFR caps, floors and SOFR swaptions
* This chapter is an overview of SOFR caps, floors and swaptions, and their pricing and heding

## Sell Side Rates Options Business

* In the rates options business, there are exotic products and vanilla products
    * Vanilla products: SOFR caps, floors and SOFR swaptions
    * Examples of exotics: Bermudan swaptions, CMS swaps, CMS caps and floors, CMS spread options, range accrual
* Exotic desk traders hedge their books by trading vanilla products
* Vanilla desk traders do so by trading underlying linear products (SOFR swaps, FRA or futures)
* More complicated models are needed for exotics pricing and hedging
    * Since exotics are hedged by vanillas, the pricing model has to give consistent pricing of both exotics and vanillas
    * Thus, the models are calibrated to vanillas

## Sell Side Rates Options Business (Cont.)

* All risk hedged out, where is the revenue from?
    * Think of zero delta ladder, portfolio PnL will be zero no matter how the market moves
    * Traders can still intentionally over/under hedge, taking directional risk, but there is in house delta limit as regulator is watching
    * Part of the revenue is from fees and bid ask spreads
* The more exotic the product is, the higher the fee and bid ask spreads, and hence the revenue
    * Justified by the significant investment in a quant team of PhDs to build and maintain the model, the hardware, and limited market players